<div style="background-color: #d9ffd4; padding: 20px; border-radius: 10px;">
    <h1 style="color: #2F4F4F; font-family: Calibri, sans-serif; text-align: center;">Jornada 4</h1>
    <p style="color: #2F4F4F; font-family: Calibri, Courier, monospace; text-align: center; font-size: 24px;">
        Simulaciones de N cuerpos: Algoritmo de Verlet y Uso de Clases en Python
    </p>
</div>

En la Jornada 3 implementamos los métodos de RK2 y RK4 para resolver EDOs de segundo orden en 2D, aplicándolos al sistema Sol–Tierra. Con el método de Euler (implementado en la Jornada 2) logramos simular la interacción gravitacional, pero detectamos que **la energía total no se conserva** debido a los errores numéricos acumulativos. Para mejorar la precisión y preservar mejor las cantidades físicas, recurrimos a los métodos de Runge–Kutta, que evalúan la pendiente en varios puntos de cada intervalo, logrando una mayor exactitud. Sin embargo, esta mejora viene acompañada de un **mayor costo computacional**, lo que abre la pregunta: ¿es posible mantener la buena conservación de energía con un algoritmo más eficiente? Aquí es donde entra en juego el **método de Verlet**.

<!-- ### **Actualización: Estudio del Método de Verlet** -->

En esta jornada incorporamos el método de Verlet, ampliamente usado en dinámica de partículas por su buena conservación de la energía en simulaciones de largo plazo y bajo costo computacional.

Compararemos su desempeño con tres métodos previos:

1. **Euler**: sencillo, pero con fuerte acumulación de errores.
2. **Runge-Kutta (RK2 y RK4)**: más precisos al evaluar pendientes intermedias.
3. **Verlet**: ofrece bajo costo computacional y una mejor conservación de la energía, aunque solo puede aplicarse a sistemas sin dependencia explícita de la velocidad, es decir, de la forma $\vec{f} = \vec{f} (t, \vec{r})$ (como ocurre en sistemas planetarios y estelares).

Nuestro objetivo es comparar estos cuatro métodos en el problema Sol–Tierra, evaluando su **costo computacional** y **precisión en la conservación de la energía mecánica**.

<div style="background-color: #d4eeff; color: black; padding: 10px; border-radius: 5px;">

### **Método de Verlet**

El **método de Verlet** es un algoritmo ampliamente utilizado para resolver ecuaciones diferenciales de **segundo orden**, en particular aquellas que describen el movimiento de partículas bajo una fuerza. Su principal ventaja es que, además de ser sencillo y eficiente, **conserva mejor la energía mecánica** en simulaciones de largo plazo.  

Partimos del sistema de ecuaciones:

$$
\frac{d^2 \vec{r}}{dt^2} = \vec{f}(t, \vec{r}),
\qquad
\vec{v} = \frac{d\vec{r}}{dt}
$$

donde $\vec{r}$ es la posición, $\vec{v}$ la velocidad y $\vec{a} = \vec{f}(t, \vec{r})$ la aceleración.

El método de Verlet se implementa en tres pasos:

1. **Cálculo de la posición futura**  

   La posición en el siguiente instante $t_{n+1}$ se aproxima como:  

   $$
   \vec{r}_{n+1} = \vec{r}_n + h \vec{v}_n + \frac{h^2}{2} \vec{a}_n
   $$

   donde $h$ es el paso temporal y $\vec{a}_n = \vec{f}(t_n, \vec{r}_n)$.

2. **Actualización de la aceleración**  

   Usamos la nueva posición para calcular la aceleración en $t_{n+1}$:  

   $$
   \vec{a}_{n+1} = \vec{f}(t_{n+1}, \vec{r}_{n+1})
   $$

3. **Actualización de la velocidad**  

   Finalmente, la velocidad en $t_{n+1}$ se obtiene promediando las aceleraciones:  

   $$
   \vec{v}_{n+1} = \vec{v}_n + \frac{h}{2} \left( \vec{a}_n + \vec{a}_{n+1} \right)
   $$

---

Este esquema permite integrar el movimiento con **bajo costo computacional** y ofrece una **mejor conservación de la energía** en comparación con métodos como Euler o Runge-Kutta, lo que lo hace ideal para problemas de dinámica de partículas y simulaciones físicas de largo plazo.

</div>


In [1]:
# como es de costumbre, importamos numpy y matplotlib
import numpy as np
import matplotlib.pyplot as plt

# además importamos el módulo ode_solvers.py construido en clases anteriores
from ode_solvers import *

<div style="background-color: #FFF9AD; color: black; padding: 10px; border-radius: 5px;">

**Problema:** el problema de dos cuerpos simplificado.

Considere el sistema Sol-Tierra, talque la Tierra se encuentra inicialmente en $\vec{r}_0 = (1.47 \times 10^{11} ,0)$ m y el sol en el origen del sistema de referencia. La velocidad inicial de la tierra es de $\vec{v}_0 = (0, 30~290)$ m/s

Calcule la trayectoria de la Tierra entre $t_0=0$ días y $t_f=365$ días usando todos los métodos (con $h=1$ día) para resolver la ecuación diferencial deducida de la segunda ley de Newton y la ley de gravitación universal:

$$ \frac{d\vec{v}}{dt} = \left( -\frac{GMx}{r^3}, -\frac{GMy}{r^3}  \right)$$

$$ \frac{d\vec{r}}{dt} = \vec{v}$$

donde $G=6.672 \times 10^{-11} ~(\text{Nm}^2/\text{kg}^2)$, es la constante de gravitación universal y $M=1.9891 \times 10^{30}$ kg es la masa del sol.
</div>

### **Evaluación del costo computacional**

Determine el **tiempo de ejecución** de cada algoritmo usando el módulo `time` y muestre sus resultados finales en un `pandas.DataFrame`.

Ejemplo de uso de `time`:

In [ ]:
import time

# simulamos la ejecución de un proceso





Calculemos ahora el tiempo de ejecución de cada algoritmo al resolver el problema de dos cuerpos simplificado con un paso temporal de $h = 1$ día:

In [ ]:
# Problema de 2 cuerpos simplificado
def f_gravitational(t, r, v):
    G = 6.672e-11 # constante de gravitación universal
    M = 1.9891e30 # masa del sol

    r_mag = np.sqrt(r[0]**2 + r[1]**2)

    return -G*M / r_mag**3 * np.array([r[0], r[1]])

# Parámetros del problema
t0 = 0  # s
tf = 365 * 24 * 3600  #s
h = 1 * 1 * 24 * 3600 #s
r0 = (1.47e11, 0) # m
v0 = (0, 30290)   # m/s

# Métodos
methods = [
    euler_method_second_order_2D,
    rk2_method_second_order_2D,
    rk4_method_second_order_2D,
    verlet_method_second_order_2D
]

# creamos una lista vacía para guardar resultados
results = []

# iteramos para calcular el tiempo de ejecución de cada método
for method in methods:
    ## calcule el tiempo de ejecución de cada algotimo
    

La estructura de datos anterior es totalmente compatible con la librería `pandas` y su clase `DataFrame`. En este caso, cada diccionario de la lista se interpreta como una fila, donde las *claves* pasan a ser los nombres de las columnas y los *valores* se asignan a las celdas correspondientes. Cuando las claves se repiten entre diccionarios, `pandas` las unifica en una misma columna.

In [ ]:
# usaremos pandas para manejo de datos tabulares y análisis estadísticos
import pandas as pd

df = pd.DataFrame(data=results)
df

In [ ]:
# ordenamos
df.sort_values(by="execution time (ms)")

El tiempo de ejecución de un algoritmo no es constante; varía cada vez que repetimos la simulación. Esta variación se debe a que, aunque el algoritmo que implementamos sea determinista, el entorno en el que se ejecuta introduce fuentes de aleatoriedad y fluctuaciones.

Entre las principales causas técnicas se encuentran:

- **Planificación del sistema operativo (scheduler):** la CPU va alternando entre distintos procesos y asigna tiempo de ejecución según prioridades, lo que genera pequeñas demoras.

- **Caché de memoria y jerarquía de memoria:** si los datos o instrucciones están en caché, la ejecución es más rápida; de lo contrario, se accede a memoria RAM, lo cual es más lento.

- **Carga del sistema:** la presencia de otros programas o servicios en ejecución compite por recursos (CPU, RAM, disco), afectando los tiempos.

- **Interpretación en Python:** Python añade sobrecarga al ejecutar funciones, manejar objetos y recolectar basura, lo que también puede introducir variabilidad.

- **Aleatoriedad de bajo nivel:** interrupciones de hardware, procesos de entrada/salida (I/O) y otros factores del entorno pueden influir.

Para mitigar estos efectos y obtener una medida más representativa del rendimiento de cada método, recurrimos a una técnica simple pero efectiva: repetir el experimento muchas veces y luego aplicar estadística descriptiva (promedios, desviaciones estándar, percentiles, etc.). De esta manera, podemos separar el comportamiento típico del método de las fluctuaciones introducidas por el entorno.

En nuestro caso, repetiremos la medición del tiempo de ejecución unas 200 veces, lo que nos permitirá estimar no solo el valor medio, sino también la dispersión de los resultados.


In [ ]:
import pandas as pd
import time

# Número de repeticiones
n_repetitions = 200

# Lista donde guardaremos los resultados
results = []

for i in range(n_repetitions):  # bucle externo
    dictionary = {}  

    for method in methods:  # bucle interno
        start_time = time.time()

        # Ejecuta el método
        method(f=f_gravitational, t0=t0, r0=r0, v0=v0, tf=tf, h=h)

        # Tiempo en ms
        exec_time = (time.time() - start_time) * 1000

        # Guardamos en el diccionario con el nombre del método
        dictionary[method.__name__] = exec_time

    # Guardamos este resultado en la lista
    results.append(dictionary)

# Convertimos la lista de diccionarios a DataFrame
df = pd.DataFrame(results)

# Renombramos columnas (opcional, si quieres nombres más cortos)
df = df.rename(columns={
    "euler_method_second_order_2D": "euler",
    "rk2_method_second_order_2D": "rk2",
    "rk4_method_second_order_2D": "rk4",
    "verlet_method_second_order_2D": "verlet"
})

# Ajustamos decimales
df = df.round(decimals=2)

df


Ahora generamos un gráfico:

In [ ]:
## grafico

El método `describe()` realiza la estadística descriptiva base de nuestra tabla de datos:

In [ ]:
# use describe

Seleccionamos promedio y desviación estándar:

In [ ]:
## seleccione solo promedio y desviación estándar


Trasponemos y ordenamos:

In [ ]:
##


Realicemos un gráfico de barras:

In [ ]:
# grafico de barras


Finalmente generamos un tabla de datos que contenga el promedio del tiempo de ejecución de cada método:

In [ ]:
# resultado esperado

,method,execution time (ms)
0,euler,3.89
1,rk2,10.30
2,rk4,26.19
3,verlet,5.31


### **Conservación de la energía como criterio de precisión**

Ahora analizaremos la **precisión** de cada algoritmo, utilizando el **criterio de conservación de la energía**.

<div style="background-color: #FFF9AD; color: black; padding: 10px; border-radius: 5px;">

**Análisis Energético:** Calcule la energía mecánica $E$ de la tierra:

$$E = K + U = \frac{1}{2}mv^2 - \frac{GmM}{r}$$

en el instante inicial $t_0$ y final $t_f$, para los cuatro métodos.

Determine el cambio relativo porcentual de energía definido como:

$$\Delta E_\% = \left | \frac{ E_\text{final} - E_\text{inicial}}{E_\text{inicial}} \right | \times 100$$

**¿Quién conserva mejor la energía?**

La masa de la tierra es $m=5.972 \times 10^{24}$ kg.
</div>

In [ ]:
# Define la función para calcular la energía
def calculate_energy(r, v):
    G = 6.672e-11  # constante de gravitación universal
    M = 1.9891e30  # masa del sol en kg
    m = 5.972e24   # masa de la tierra en kg

    K = 0.5 * m * np.linalg.norm(v, axis=1)**2  # energía cinética
    U = - G * m * M / np.linalg.norm(r, axis=1)  # energía potencial
    E = K + U  # energía total

    return E

# Itera sobre cada método, ejecuta la simulación y calcula los cambios porcentuales de energía

# resultado esperado

,method,execution time (ms),energy changes (%)
0,euler,3.89,1.524116e+01
1,rk2,10.30,4.044385e-04
2,rk4,26.19,2.677552e-08
3,verlet,5.31,5.213373e-09


Ordenamos la tabla primero, por los cambios porcentuales de energía y segundo, por el tiempo de ejecución del algoritmo:

In [ ]:
## 

Acá, por el contrario, ordenamos la tabla primero, por el tiempo de ejecución del algoritmo y segundo, por los cambios porcentuales de energía:

In [ ]:
##  

#### **Gráfico de Dispersión tipo Trade-off**

A continuación, vamos a construir un **gráfico de dispersión tipo trade-off**, donde cada punto representa a un algoritmo, mostrando en un eje su **tiempo de ejecución** y en el otro el **cambio relativo en la energía mecánica**. De esta manera, podremos visualizar claramente qué métodos son más rápidos y cuáles conservan mejor la energía.

La elección del algoritmo dependerá de las prioridades:

- Si buscamos máxima precisión física, preferiremos aquel con menor variación de energía.
- Si el objetivo es eficiencia computacional, elegiremos el de menor tiempo de ejecución.
- Y si queremos un balance entre velocidad y precisión, seleccionaremos el método que se ubique más cerca del *punto ideal* (tiempo bajo y buena conservación de energía)

In [ ]:
## cree el gráfico de dispersión (scatter plot)


Según los resultados obtenidos, el **método de Verlet** ofrece el mejor compromiso entre tiempo de ejecución y precisión en la conservación de la energía, siempre que se trate de sistemas donde las ecuaciones de movimiento no dependen explícitamente de la velocidad. Este tipo de situaciones corresponde, justamente, a los **sistemas conservativos**, como los que describen la interacción gravitacional entre cuerpos.

Por esta razón, Verlet es especialmente útil en **simulaciones de N cuerpos gravitacionales**, donde su bajo costo computacional y su capacidad para preservar la energía lo convierten en una herramienta ideal para estudios de largo plazo.

En contraste, en sistemas **no conservativos**, es decir, aquellos con fuerzas disipativas que sí dependen de la velocidad, el método de Verlet deja de ser adecuado.

<div style="background-color: #FFF9AD; color: black; padding: 10px; border-radius: 5px;">

**Tarea: Probar algoritmo de verlet para el problema de movimiento parabólico con roce**

</div>

## **Simulaciones de N Cuerpos**

Una simulación de N cuerpos es un problema clásico en la astrofísica que consiste en resolver las ecuaciones de movimiento de múltiples cuerpos celestes que interactúan entre sí a través de la fuerza de gravedad. En este contexto, cada cuerpo puede ser una estrella, un planeta o cualquier otro objeto masivo en el universo.

A continuación, implementaremos un enfoque basado en estructuras de clases para organizar la información sobre cada cuerpo celeste y la simulación en sí misma. Esto nos permitirá gestionar mejor los datos y facilitar la implementación de algoritmos para calcular las trayectorias de los cuerpos.

### **Definición de Clases en Python**

#### **Clase CelestialBody**

La clase `CelestialBody` modela un cuerpo celeste individual dentro de una simulación gravitacional. Su propósito es encapsular las propiedades físicas fundamentales de cada objeto astronómico y proveer las herramientas necesarias para actualizar su estado dinámico en el tiempo.

```python
class CelestialBody:
    def __init__(self, mass, position, velocity, name):

        self.mass = mass
        self.init_position = np.array(position, dtype=float)
        self.init_velocity = np.array(velocity, dtype=float)

        self.name = name

        # Arrays to store positions, velocities, and accelerations will be initialized later
        self.positions = None
        self.velocities = None
        self.accelerations = None

        # Keep track of current step in the simulation
        self.current_step = 0
```

Llamamos a la clase `CelestialBody` desde el módulo `nbody_simulation_tools`:

In [ ]:
from nbody_simulation_tools import CelestialBody

earth = ??
moon = ??

#### **Clase NBodySimulation**

La clase `NBodySimulation` está diseñada para gestionar de manera integral la dinámica de un sistema compuesto por múltiples cuerpos celestes que interactúan gravitacionalmente. Su función principal es coordinar la evolución temporal del sistema aplicando las leyes de la mecánica newtoniana y los algoritmos numéricos de integración elegidos.

#### **Planteamiento del cálculo de fuerzas**

<p align="center">
  <img src="nbody_interactions.png" style="width: 40%; height: auto;">
</p>

Observe el esquema anterior. El cuerpo de masa $m_i$ experimentará una fuerza $\vec{F}_{ji}$ ejercida por el cuerpo de masa $m_j$. Según la ley de gravitación universal de Newton, dicha fuerza se calcula como:

$$ \vec{F}_{ji} = -G \frac{m_j m_i}{{r}_{ji}^2} \hat{r}_{ji} = -G \frac{m_j m_i}{|\vec{r}_{ji}|^3} \vec{r}_{ji} = -G \frac{m_j m_i}{|\vec{r}_i - \vec{r}_j|^3} ( \vec{r}_i - \vec{r}_j) $$

Ahora, la fuerza total que experimentará $m_i$ por la acción de las fuerzas del resto de $N-1$ cuerpos de masas $m_j$ será la sumatoria de todas las fuerzas:

$$ \vec{F}_i = \sum_{j=1}^N \vec{F}_{ji} = \sum_{j=1}^N -G \frac{m_j m_i}{|\vec{r}_i - \vec{r}_j|^3} ( \vec{r}_i - \vec{r}_j)~~,~~j\neq i  $$

La aceleración que experimentará $m_i$ por la acción de estas fuerzas será:

$$ \vec{a}_i = \frac{\vec{F}_i}{m_i} = \sum_{j=1}^N \frac{-Gm_j}{|\vec{r}_i - \vec{r}_j|^3} ( \vec{r}_i - \vec{r}_j)~~,~~j\neq i  $$

Ahora podemos determinar el vector posición $\vec{r_i}$ y su respectivo vector velocidad $\vec{v}_i$, al cabo de un paso de tiempo $dt$ usando el algoritmo de Verlet. Usaremos la notación $\vec{r}$ y $\vec{v}$ para ahorrar escritura:

1. **Posición futura**:
   
   La posición en el siguiente paso de tiempo, $\vec{r}_{n+1}$, se aproxima con:

   $$
   \vec{r}_{n+1} = \vec{r}_n + dt \cdot \vec{v}_n + \frac{dt^2}{2} \vec{a}_n 
   $$

   donde $\vec{a}_n$ es la aceleración que experimenta $m_i$ en el instante $t_n$, debido a la acción de las múltiples fuerzas.

2. **Aceleración actualizada**:
   
   Calculamos la aceleración $\vec{a}_{n+1}$ usando el nuevo vector $\vec{r}_{n+1}$ y el nuevo tiempo $$ t_{n+1} = t_n + dt $$

3. **Velocidad actualizada**:
   
   La velocidad en el nuevo paso se calcula utilizando la aceleración actualizada:

   $$
   \vec{v}_{n+1} = \vec{v}_n + \frac{dt}{2} \left( \vec{a}_n + \vec{a}_{n+1} \right)
   $$


Entonces, primero que todo, debemos crear un método que permita calcular la fuerza neta que experimenta un cuerpo de masa $m_i$ por la acción del resto de cuerpos, ya que es necesario para luego determinar la aceleración, velocidad y posición de cada cuerpo, con el algoritmo de Verlet.

```python
class NBodySimulation:
    def __init__(self, dt: float, num_steps: int) -> None:

        self.bodies = {}
        self.dt = dt
        self.num_steps = num_steps

        self.G = 6.67430e-11  # universal gravitational constant (m^3 kg^-1 s^-2)

        self.current_step = 0


    def add_body(self, body):
        """Add a CelestialBody to the simulation and initialize arrays."""
        self.bodies[body.name] = body
        body._initialize_arrays(self.num_steps)

    def compute_gravitational_forces(self, delta=1e4):
        pass

    def update(self):
        pass

    def run_simulation(self):
        pass
```

In [4]:
from nbody_simulation_tools import NBodySimulation

# parametros de simulación
dt = 1 * 3600 * 24  # paso de tiempo de un día
num_steps = 365  # 365 pasos de un día es igual a un año de simulación

simulation = NBodySimulation(dt=dt, num_steps=num_steps)

simulation.bodies

{}

Tenemos un objeto `simulation` pero no contiene ningún cuerpo celeste, debemos añadirlos con el método `add_body` de la siguiente manera:

In [ ]:
## añada los cuerpos


Ahora podemos acceder a ellos desde `simulation` y extraer sus parámetros:

In [ ]:
# masa de la Tierra



In [ ]:
# Posiciones de la Luna


Notar que el array de posiciones de la luna está lleno de ceros, esto significa que la simulación aún no ha comnezado. Para iniciarla podemos usar `run_simulation()`:

In [ ]:
# Ejecutar la simulación


Ahora veamos las posiciones:

In [ ]:
##

A graficar:

In [ ]:
###

La Tierra se mueve ligeramente!

#### **Centro de masa del sistema**

En un sistema de $N$ cuerpos, el centro de masa es un punto teórico que representa el promedio ponderado de las posiciones de cada cuerpo, teniendo en cuenta sus masas. Si cada cuerpo en el sistema tiene una masa diferente y ocupa una posición distinta, el centro de masa es el lugar donde, en promedio, podemos considerar que se concentra toda la masa del sistema para describir su movimiento global. El centro de masa se determina con la siguiente fórmula:

$$
\vec{r}_\text{cm} = \frac{1}{M} \sum_{i=1}^N m_i \vec{r}_i~~,~~M = \sum_{i=1}^N m_i 
$$

Donde $\vec{r}_i$ es el vector posición de cada cuerpo y $m_i$ su masa. Como los cuerpos se mueven, sus posiciones cambian con el tiempo, es decir, $\vec{r}_i = \vec{r}_i(t)$. Por lo tanto, el centro de masa también dependerá del tiempo y su posición estará en función del mismo:

$$
\vec{r}_\text{cm}(t) = \frac{1}{M} \sum_{i=1}^N m_i \vec{r}_i(t)~~,~~M = \sum_{i=1}^N m_i 
$$

Para centrar el sistema de referencia en el centro de masa, se restan las coordenadas del centro de masa a cada vector posición del cuerpo en el sistema, obteniendo así las posiciones relativas de los cuerpos respecto al centro de masa:

$$
\vec{r}_i'(t) = \vec{r}_i(t) - \vec{r}_\text{cm}(t)
$$

De esta manera, al cambiar al sistema de referencia centrado en el centro de masa, podemos observar cómo los objetos orbitan alrededor de este punto, que actúa como el "punto de equilibrio" dinámico del sistema.



El método `shift_to_center_of_mass()` calcula el centro de masa y centra los objetos en torno a él:

In [ ]:
##

Al parecer el $dt$ es muy grande, disminuyámoslo un poco:

In [ ]:
###


In [ ]:
###

Ahora centrar al centro de masa:

In [ ]:
### centrar

**¿Y si la Luna tuviera más masa, como por ejemplo `7.349e23`, que pasaría?**

In [ ]:
earth = CelestialBody(mass=5.972e24, init_position=(0, 0), init_velocity=(0, 0), name="Tierra")
moon = CelestialBody(mass=7.349e23, init_position=(3.844e8, 0), init_velocity=(0, 1000), name="Luna")

dt = 1 * 3600 # 1 hora
num_steps = 24 * 28 # periodo rotacional lunar


simulation = NBodySimulation(dt=dt, num_steps=num_steps)

simulation.add_body(earth)
simulation.add_body(moon)

simulation.run_simulation()

#simulation.shift_to_center_of_mass()

plt.plot(moon.positions[:, 0], moon.positions[:, 1], ls="--", color="gray")
plt.plot(earth.positions[:, 0], earth.positions[:, 1], ls="--")

plt.scatter(moon.positions[-1, 0], moon.positions[-1, 1], color="gray", s=10)
plt.scatter(earth.positions[-1, 0], earth.positions[-1, 1])

plt.scatter(0, 0, marker="x", color="red")
plt.grid(ls="--", alpha=.5)
plt.axis("equal")
plt.show()

In [ ]:
simulation.shift_to_center_of_mass()

plt.plot(earth.positions[:, 0], earth.positions[:, 1], ls="--")
plt.scatter(earth.positions[-1, 0], earth.positions[-1, 1])

plt.plot(moon.positions[:, 0], moon.positions[:, 1], ls="--", color="gray")
plt.scatter(moon.positions[-1, 0], moon.positions[-1, 1], color="gray", s=10)

plt.scatter(0, 0, marker="x", color="red")
plt.grid(ls="--", alpha=.5)
plt.axis("equal")
plt.show()

<div style="background-color: #FFF9AD; color: black; padding: 10px; border-radius: 5px;">

**Ejercicio: Simulación de los planetas internos del sistema solar**


Simule los planetas internos del sistema solar, busque en internet la distancia media al sol de Mercurio, Venus, Tierra y Marte, además de su velocidad media para usarla como condiciones iniciales de una simulación de $N$ cuerpos.

No olvide buscar las masas!

¿Conocen algún sitio web de donde encontrar sistemas planetarios de otras regiones del universo?

Podría usar esa información para simular dichos sistemas!

</div>

In [ ]:
## buscar datos en la web



**¿El sol se mueve?**

In [ ]:
# grafique solo la trayectoria del sol



Centrar:

In [ ]:
# centrar


<div style="background-color: #FFF9AD; color: black; padding: 10px; border-radius: 5px;">

**Ejercicio: Influencia Gravitacional de la Luna sobre la Tierra**

Realice dos simulaciones por separado, sistema Sol Tierra y Sistema Sol Tierra Luna. Calcule la magnitud de la posición de la Tierra y grafíquelas en función del tiempo. ¿Hay alguna diferencia? ¿La luna influye en la trayectoria de la Tierra?

In [ ]:
## simulación 1

In [ ]:
## simulación 2

In [ ]:
## grafico de la magnitud de la posición de la Tierra vs tiempo en ambas simulaciones



<div style="background-color: #d9ffd4; color: black; padding: 10px; border-radius: 5px;">

**Conclusión:**

En esta jornada realizamos una comparación detallada entre los métodos numéricos Euler, Runge-Kutta de segundo orden (RK2), Runge-Kutta de cuarto orden (RK4) y el método de Verlet, evaluando tanto su **precisión** como su **costo computacional** en la simulación de sistemas conservativos. Los resultados mostraron que Verlet ofrece un equilibrio óptimo: mantiene una precisión comparable a RK4 mientras requiere menor tiempo de cómputo, convirtiéndolo en una herramienta ideal para **simulaciones de cuerpos celestes y sistemas N cuerpos**, donde la **estabilidad** a largo plazo es fundamental.

Posteriormente, implementamos una estructura de clases organizada con `CelestialBody` y `NBodySimulation`, que permite gestionar de manera eficiente la información de posición, velocidad y aceleración mediante arrays de NumPy. Esta organización no solo mejora la claridad del código, sino que también optimiza la eficiencia de las simulaciones.

Llevamos a cabo simulaciones de distintos sistemas, incluyendo Tierra-Luna y el sistema solar interno, lo que nos permitió estudiar su comportamiento dinámico de manera detallada. En particular, se destacó la importancia de **centrar el sistema en el centro de masa**, lo que facilita la visualización de las órbitas y permite representar con precisión las trayectorias de los cuerpos celestes.

En conjunto, estas actividades nos permitieron profundizar en la simulación de sistemas N cuerpos, optimizando el balance entre precisión y eficiencia computacional, y destacando la relevancia de métodos adecuados para la conservación de la energía en sistemas conservativos.

**Preguntas abiertas:** ¿Cómo se conserva la energía mecánica en estas simulaciones? ¿Cómo se calcula y qué factores pueden afectar su conservación?


</div>

<div style="padding: 15px; border-top: 2px solid #2F4F4F; margin-top: 30px; background-color: var(--custom-bg-color); color: var(--custom-text-color);">
    <p style="font-family: Calibri, sans-serif; text-align: left; font-size: 16px;">
        Omar Fernández <br>
        Profesor de Física Computacional IV para Astrofísica <br>
        Ingeniero Físico <br>
        <a href="mailto:omar.fernandez.o@usach.cl" class="email-link">omar.fernandez.o@usach.cl</a> <br>
    </p>
</div>

<style>
:root {
    --custom-bg-color: #F8F8F8;
    --custom-text-color: #2F4F4F;
    --custom-link-color: blue;
}

@media (prefers-color-scheme: dark) {
    :root {
        --custom-bg-color: #444444;
        --custom-text-color: #F8F8F8;
        --custom-link-color: magenta;
    }
}

.email-link {
    color: var(--custom-link-color);
}
</style>